## Using Komo for IK

In [ ]:
import sys
sys.path.append('../../build')
import numpy as np
import libry as ry

In [ ]:
# Here we do not need a simulation world
# adding a configuration world
C = ry.Config()
C.addFile("../../scenarios/pandasTable.g")
V = ry.ConfigurationViewer()
V.setConfiguration(C)

In [ ]:
obj = C.addFrame("object")
obj.setPosition([.8,0,1.5])
obj.setQuaternion([1,0,1,0])
obj.setShape(ry.ST.capsule, [.2,.02])
obj.setColor([1,0,1])
V.setConfiguration(C)

In [ ]:
R_gripper = C.frame("R_gripper")
R_gripper.setContact(1)
obj.setContact(1)

In [ ]:
IK = C.komo_IK(False)
IK.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e2])
IK.addObjective([1.], ry.FS.positionDiff, ["R_gripperCenter", "object"], ry.OT.sos, [1e2]);

In [ ]:
# Calling the optimizer (True means random initialization/restart)
IK.optimize()
IK.getReport()

In [ ]:
C.setFrameState( IK.getConfiguration(0) )
V.setConfiguration(C) #to update your model display

In [ ]:
# Move object and reoptimize

# move object
obj.setPosition([.2,.2,1.5])

# copy C into the IK's internal configuration(s)
IK.setConfigurations(C)

# reoptimize
IK.optimize(0.) # 0 indicates: no adding of noise for a random restart
print(IK.getReport())

# grab result
C.setFrameState( IK.getConfiguration(0) )
V.setConfiguration(C) #to update your model display

## Path Optimization

In [ ]:
obj.setPosition([.8,.2,1.5])

# we want to optimize a single step (1 phase, 1 step/phase, duration=1, k_order=1)
komo = C.komo_path(1.,20, 5., True)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e2])
komo.addObjective([1.], ry.FS.positionDiff, ["R_gripperCenter", "object"], ry.OT.sos, [1e2])
komo.optimize()
komo.getReport()

In [ ]:
V = komo.view()

In [ ]:
V.playVideo()

In [ ]:
C.setFrameState(komo.getConfiguration(19))
C.getJointState()
V.setConfiguration(C)
coll = C.feature(ry.FS.accumulatedCollisions, [])
C.computeCollisions() 
coll.eval(C)